In [10]:
# @title Инициализация Google Earth Engine
import ee
import geemap
Map = geemap.Map()
Map


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [11]:
# @title Ввод исходных данных и их первичная обработка (введите исходный ассет)
i = "projects/ee-alexpuh80/assets/39_69_1_1_2m_v41_dem" # @param {type:"string"}


point = ee.Geometry.Point([41.9108, 64.0984])
#point = ee.Geometry.Point([-87.7719, 41.8799])

image = ee.Image(i)


d = ee.Image('projects/ee-alexpuh80/assets/39_69_1_1_2m_v41_dem').int16()

hydro = ee.Terrain.fillMinima(d, -200, 1000)

dem = ee.Image("MERIT/Hydro/v1_0_1").select('elv').rename(['b1']).clip(d.geometry()).int16()

image1 = ee.Image('projects/ee-alexpuh80/assets/trainings').select('b1')

dem1 = ee.Image('projects/ee-alexpuh80/assets/40_71_32m_v41_dem').int16()

hydro1 = ee.Terrain.fillMinima(dem1, -200, 0)

region = Map.user_roi

r = ee.Image("MERIT/Hydro/v1_0_1").select('elv').rename(['b1']).clip(region)

r1=r.int16()

hydro2 = ee.Terrain.fillMinima(r1, -5000, 0)

region2 = ee.Geometry.Rectangle([104.8, 53.28, 141.032, 73.115])

re = ee.Image("MERIT/Hydro/v1_0_1").select('elv').rename(['b1']).clip(region2).int16()

hydro3 = ee.Terrain.fillMinima(re, -200, 0)

vis_params = {
    'min': 10,
    'max': 250,
    'bands': ['b1'],
    'palette': ['0D28EC','0DEC5E', 'DBEC0D', 'EC7D0D','EC360D']
}

vis_params1 = {
    'min': 10,
    'max': 250,
    'bands': ['b1'],
    'palette': ['0D28EC','0DEC5E', 'DBEC0D', 'EC7D0D','EC360D']
}

Map.centerObject(hydro, 8)
Map.addLayer(image, vis_params, "DEM")
# Map.addLayer(image1, vis_params, "DEM2")
Map.addLayer(hydro, vis_params, "Improved DEM")
# Map.addLayer(hydro1, vis_params, "Improved DEM2")
# Map.addLayer(hydro2, vis_params, "Region")
# Map.addLayer(hydro3, vis_params1, "Training Region")

In [12]:
# @title Ввод данных для обучения (введите ассет для обучения и его группу)
n = "ESA/GLOBCOVER_L4_200901_200912_V2_3" # @param {type:"string"}
band = "landcover" # @param {type:"string"}

nlcd = ee.Image(n).select(band).clip(hydro.geometry())
#nlcd1 = ee.Image('ESA/GLOBCOVER_L4_200901_200912_V2_3').select('landcover').clip(hydro1.geometry())
#nlcd2 = ee.Image('ESA/GLOBCOVER_L4_200901_200912_V2_3').select('landcover').clip(hydro2.geometry())
#nlcd3 = ee.Image('ESA/GLOBCOVER_L4_200901_200912_V2_3').select('landcover').clip(hydro3.geometry())
#nlcdtest = ee.Image('ESA/GLOBCOVER_L4_200901_200912_V2_3').select('landcover').clip(flowAccumulation.geometry())
#nlcdf = ee.Image('MERIT/Hydro/v1_0_1').select('upa').int16().clip(hydro.geometry())

#nlcdf = ee.Image('ESA/GLOBCOVER_L4_200901_200912_V2_3').select('landcover').clip(hydro2.geometry())
Map.addLayer(nlcd, {}, 'NLCD')
#Map.addLayer(nlcd1, {}, 'NLCD1')
#Map.addLayer(nlcd2, {}, 'NLCD2')
#Map.addLayer(nlcd3, {}, 'NLCD3')
#Map.addLayer(nlcdf, {}, 'NLCDT')
Map

Map(bottom=17655.0, center=[64.2943881859635, 41.45523354230649], controls=(WidgetControl(options=['position',…

In [20]:
# @title Создание рабочего датасета для обучения (введите количество прецедентов)
precedents = 500 # @param {type:"number"}
points = nlcd.sample(**{
     'region': hydro.geometry(),
     'scale': 30,
     'numPixels': precedents,
     'seed': 0,
     'geometries': True  # Set this to False to ignore geometries
})

# points1 = nlcd3.sample(**{
#     'region': hydro3.geometry(),
#     'scale': 30,
#     'numPixels': 100000,
#     'seed': 0,
#     'geometries': True  # Set this to False to ignore geometries
# })

# points2 = nlcd1.sample(**{
#     'region': hydro1.geometry(),
#     'scale': 30,
#     'numPixels': 100000,
#     'seed': 0,
#     'geometries': True  # Set this to False to ignore geometries
# })

# points4 = nlcdtest.sample(**{
#     'region': hydro1.geometry(),
#     'scale': 30,
#     'numPixels': 100000,
#     'seed': 0,
#     'geometries': True  # Set this to False to ignore geometries
# })
# points5 = nlcdf.sample(**{
#     'region': hydro3.geometry(),
#     'scale': 30,
#     'numPixels': 1000,
#     'seed': 0,
#     'geometries': True  # Set this to False to ignore geometries
# })

points = nlcd.sample(**{
    'region': hydro.geometry(),
    'scale': 30,
    'numPixels': 500,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})

sample = ee.FeatureCollection('projects/ee-alexpuh80/assets/SamplePoints')#.merge(Map.user_rois)
sample2 = ee.FeatureCollection('projects/ee-alexpuh80/assets/SamplePoints2')#.merge(Map.user_rois)

#points = nlcd.sample(sample2)

validation = sample.merge(sample2)
Map.addLayer(points, {}, 'training', True)
#Map.addLayer(points1, {}, 'training1', False)
#Map.addLayer(points2, {}, 'training2', False)
#Map.addLayer(points4, {}, 'training3', False)
#Map.addLayer(points5, {}, 'training5', False)
#Map.addLayer(sample, {}, 'sample', True)
#Map.addLayer(sample2, {}, 'sample2', True)

In [21]:
# @title Проверка №1
print(points.size().getInfo())

500


In [22]:
# @title Проверка №2
print(points.first().getInfo())

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [41.097011341553696, 64.3331948451675]}, 'id': '0', 'properties': {'landcover': 110}}


In [55]:
# @title Обучение ИИ-метода (введите исходную группу, группу прецедентов, класссификатор и его параметры)
b = "b1" # @param {type:"string"}
l = "landcover" # @param {type:"string"}
classifier = "KNN" # @param {type:"string"}
parameters = 15 # @param {type:"raw"}


# Use these bands for prediction.
bands = [b]

# This property of the table stores the land cover labels.
label = l

# # Overlay the points on the imagery to get training.
# training = hydro.select(bands).sampleRegions(**{
#   'collection': points5,
#   'properties': [label],
#   'scale': 30
# })

# retraining = hydro3.select(bands).sampleRegions(**{
#   'collection': points5,
#   'properties': [label],
#   'scale': 30
# })
# finaltraining = hydro1.select(bands).sampleRegions(**{
#   'collection': points5,
#   'properties': [label],
#   'scale': 30
# })
flowtraining = hydro.select(bands).sampleRegions(**{
  'collection': points,
  'properties': [label],
  'scale': 15
})

# Train a CART classifier with default parameters.
if (classifier == "Cart"):
  trained = ee.Classifier.smileCart(parameters).train(flowtraining, label, bands)
if (classifier == "GTB"):
  trained = ee.Classifier.smileGradientTreeBoost(parameters).train(flowtraining, label, bands)
if (classifier == "MD"):
  trained = ee.Classifier.minimumDistance(parameters).train(flowtraining, label, bands)
if (classifier == "RF"):
  trained = ee.Classifier.smileRandomForest(parameters).train(flowtraining, label, bands)
if (classifier == "KNN"):
  trained = ee.Classifier.smileKNN(parameters).train(flowtraining, label, bands)
# retrained = trained.smileCart().train(retraining, label, bands)
# finaltrained = retrained.smileCart().train(finaltraining, label, bands)
#flowtrained = ee.Classifier.smileCart().train(flowtraining, label, bands)
#flowtrained = ee.Clusterer.wekaLVQ(10).train(flowtraining, ['b1'], 1, 0)
# flowtrained = ee.Clusterer.wekaKMeans(20).train(flowtraining, ['b1'], 1, 0)
#flowtrained = ee.Classifier.confusionMatrix(finaltrained)


In [47]:
# @title Проверка набора
print(flowtraining.first().getInfo())



{'type': 'Feature', 'geometry': None, 'id': '0_0', 'properties': {'b1': 12, 'landcover': 110}}


In [56]:
# @title Первичная классификация
# Classify the image with the same bands used for training.
# result = hydro2.select(bands).classify(flowtrained)
result2=hydro.select(bands).classify(trained)
# combined_img = ee.ImageCollection([result, result2]).mosaic()

# flowresult=hydro.select(bands).cluster(flowtrained)

# # Display the clusters with random colors.
#Map.addLayer(combined_img, {}, 'classfied')
#Map.addLayer(result, {}, 'classfied')
Map.addLayer(result2.randomVisualizer(), {}, 'classfied')
Map.centerObject(result2, 9)
#Map.addLayer(flowresult.randomVisualizer(), {}, 'classfied1')
#Map.addLayer(flowresult.randomVisualizer(), {}, 'classfied1')
Map

Map(bottom=35010.0, center=[64.2943881859635, 41.45523354230649], controls=(WidgetControl(options=['position',…

In [49]:
# @title Определение реки

#class_values = nlcd.get('landcover_class_values').getInfo()
class_values = [#11,
                # 14,
                # 20,
                # 30,
                # 40,
                # 50,
                # 60,
                # 70,
                # 90,
                # 100,
                # 110,
                # 120,
                # 130,
                # 140,
                # 150,
                # 160,
                # 170,
                # 180,
                # 190,
                # 200,
                210,
                # 220,
                # 230
                ]
class_values

#class_palette = nlcd.get('landcover_class_palette').getInfo()

class_palette = [
# 'aaefef',
# '000000',
# 'dcef63',
# 'cdcd64',
# '006300',
# '009f00',
# 'aac700',
# '003b00',
# '286300',
# '788300',
# '8d9f00',
# 'bd9500',
# '956300',
# 'ffb431',
# 'ffebae',
# '00785a',
# '009578',
# '00dc83',
# 'c31300',
# 'fff5d6',
'0046c7',
# 'ffffff',
# '743411'
]
class_palette

landcover = result2.set('classification_class_values', class_values)
landcover = landcover.set('classification_class_palette', class_palette)

flow = result2.set('classification_class_values', class_values)
flow = flow.set('classification_class_palette', class_palette)

final = ee.Image(flow)
Map.addLayer(final,{},'RiverA')
Map

Map(bottom=35010.0, center=[64.29467520380601, 41.45141601562501], controls=(WidgetControl(options=['position'…

In [43]:
# @title Векторизация реки (введите максимальное количество точек)
max = 1000 # @param {type:"number"}
projection=final.projection();
scale=final.projection().nominalScale();

# Vector=final.reduceToVectors(**{
#     'geometry': hydro.geometry(),
#     'crs': projection,
#     'scale': scale,
#     'geometryType': 'polygon',
#     'eightConnected': False,
#    'labelProperty': 'label',
#     'bestEffort': False,
#     'maxPixels': 1e13,
#     'tileScale': 3,
#  });

pointst = final.stratifiedSample(**{
    'region': hydro.geometry(),
    'projection': projection,
    'scale': scale,
    'numPoints': 1000,
    'seed': 0,
    'geometries': True,
    'dropNulls': False,
    'tileScale': 3,
    'classValues':[11,14,20,30,40,50,60,70,90,100,110,120,130,140,150,160,170,180,190,200,210,220,230],
    'classPoints':[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,max,0,0]
})

#print(pointst.first().getInfo())

Drawn=pointst.draw(**{
   'color': 'ff0000',
   'strokeWidth': 1
});
Map.addLayer(Drawn,{}, 'Vector');
#Map.addLayer(Vector,{}, 'Vector1');
#Map.addLayer(Drawn,{}, 'Drawn');

In [44]:
# @title Экспорт векторного результата (введите имя экспортируемого файла)
name = "TestPoints" # @param {type:"string"}
!pip install -U -q earthengine-api
!pip install -U -q folium
!pip install -U -q git+https://github.com/mccarthyryanc/folium_gee.git

!pip install geopandas
!pip install descartes
!pip install geemap

!pip install -U -q git+https://github.com/nicolas998/WMF.git@develop

!pip install -U numba
!pip install pysheds

import folium
import foliumgee
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from geopy.distance import geodesic
from scipy.spatial.distance import cdist
from geopy.distance import distance as geodist # avoid naming confusion
import subprocess
import os
import multiprocessing as mp
import numpy as np
from pprint import pprint
from google.colab import files
#from datetime import datetime
from datetime import datetime
from datetime import timedelta
from datetime import date
import time
from dateutil.parser import parse
try:
        import geemap
except ImportError:
        print('geemap package not installed. Installing ...')
        subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
        import google.colab
        import geemap.eefolium as emap
except:
        import geemap as emap


from google.colab import drive
drive.mount('/content/drive')

try:
  ee.Initialize()
  print('Google Earth Engine has initialized successfully!')
except ee.EEException as e:
  print('Google Earth Engine has failed to initialize!')
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise

# Define the URL format used for Earth Engine generated map tiles.
EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'

def Mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = v["tile_fetcher"].url_format,
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz


task = ee.batch.Export.table.toDrive(**{
  'collection': pointst,
  'description': name,
  'folder': 'Example_folder',
  'fileFormat': 'SHP',
})
task.start()

import time
while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(5)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.0/385.0 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.4/108.4 kB 1.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 MB 11.4 MB/s eta 0:00:00
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.41.1
    Uninstalling llvmlite-0.41.1:
      Successfully uninstalled llvmlite-0.41.1
  Attempting uninstall: numba
    Found existing installation: numba 0.58.1
    Uninstalling numba-0.58.1:
      Successfully uninstalled numba-0.58.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 1.5.3 which is incompatible.
     ━